## **Double Lasso - Using School data**

In [132]:
import Pkg 
Pkg.add("DataFrames")
Pkg.add("CausalInference")
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("Lasso")
Pkg.add("HDMjl")
Pkg.add("DataStructures")
Pkg.add("NamedArrays")
Pkg.add("PrettyTables")
Pkg.add("Distributions")
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, DataStructures, NamedArrays, PrettyTables, CSV, Plots, Dates, HDMjl


   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Alvaro\.julia\environments\v1.11\Project.

In [115]:
#Reading CSV file
data = CSV.read("data/bruhn2016.csv", DataFrame)
first(data,10)

Row,outcome.test.score,treatment,school,is.female,mother.attended.secondary.school,father.attened.secondary.school,failed.at.least.one.school.year,family.receives.cash.transfer,has.computer.with.internet.at.home,is.unemployed,has.some.form.of.income,saves.money.for.future.purchases,intention.to.save.index,makes.list.of.expenses.every.month,negotiates.prices.or.payment.methods,fincial.autonomy.index
,Float64,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,47.3674,0,17018390,missing,missing,missing,missing,missing,missing,1,1,0,29,0,1,52
2,58.1768,1,33002614,missing,missing,missing,missing,missing,missing,0,0,0,41,0,0,27
3,56.6717,1,35002914,1,1,1,0,0,0,1,0,0,48,0,1,56
4,29.0794,0,35908915,1,0,0,0,0,0,0,0,0,42,0,0,27
5,49.5635,1,33047324,1,0,0,0,0,1,0,1,0,50,0,1,31
6,43.0994,0,53012542,1,0,0,0,1,1,0,1,0,93,0,1,77
7,71.3296,0,53006984,1,1,0,0,0,1,1,1,0,52,0,1,43
8,80.0634,0,33049475,1,1,1,0,0,0,0,1,0,50,0,0,31
9,88.7516,1,35925639,1,1,0,0,0,0,0,0,0,59,0,1,50


In [116]:
#Los nombres de las variables hacen clash con las originales de CSV, asi que las cambiamos
outcome = data[!,"outcome.test.score"]
outcome = DataFrame([outcome],[:outcome])
treatment = data[!,"treatment"]
treatment = DataFrame([treatment],[:treatment])
#Codigo Para Convetir los valores de escuela en ID unica
school = data[!,"school"]
c = unique(school)
cd = eachrow(c)
d = size(school)[1]
for i in c
    row_in_c = findall(x -> x == i, c)[1]
    school[school.==i] .= row_in_c
end
school = DataFrame([school],[:school])
female = data[!,"is.female"]
female = DataFrame([female],[:female])
mother_secondary = data[!,"mother.attended.secondary.school"]
mother_secondary = DataFrame([mother_secondary],[:mother_secondary])
father_secondary = data[!,"father.attened.secondary.school"]
father_secondary = DataFrame([father_secondary],[:father_secondary])
failed_one_year = data[!,"failed.at.least.one.school.year"]
failed_one_year = DataFrame([failed_one_year],[:failed_one_year])
cash_transfer = data[!,"family.receives.cash.transfer"]
cash_transfer = DataFrame([cash_transfer],[:cash_transfer])
computer = data[!,"has.computer.with.internet.at.home"]
computer = DataFrame([computer],[:computer])
financial_index = data[!,"fincial.autonomy.index"]
financial_index = DataFrame([financial_index],[:financial_index])
data = [outcome treatment school female mother_secondary father_secondary failed_one_year cash_transfer computer]
first(data,11)

Row,outcome,treatment,school,female,mother_secondary,father_secondary,failed_one_year,cash_transfer,computer
,Float64,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,47.3674,0,1,missing,missing,missing,missing,missing,missing
2,58.1768,1,2,missing,missing,missing,missing,missing,missing
3,56.6717,1,3,1,1,1,0,0,0
4,29.0794,0,4,1,0,0,0,0,0
5,49.5635,1,5,1,0,0,0,0,1
6,43.0994,0,6,1,0,0,0,1,1
7,71.3296,0,7,1,1,0,0,0,1
8,80.0634,0,8,1,1,1,0,0,0
9,88.7516,1,9,1,1,0,0,0,0


In [157]:
data = dropmissing(data, :female)
data = dropmissing(data, :computer)
data = dropmissing(data, :school)
data = dropmissing(data, :father_secondary)
data = dropmissing(data, :cash_transfer)
data = dropmissing(data, :failed_one_year)
data = dropmissing(data, :mother_secondary)
data = dropmissing(data, :treatment)
data = dropmissing(data, :outcome)

first(data,10)

Row,outcome,treatment,school,female,mother_secondary,father_secondary,failed_one_year,cash_transfer,computer
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,56.6717,1,3,1,1,1,0,0,0
2,29.0794,0,4,1,0,0,0,0,0
3,49.5635,1,5,1,0,0,0,0,1
4,43.0994,0,6,1,0,0,0,1,1
5,71.3296,0,7,1,1,0,0,0,1
6,80.0634,0,8,1,1,1,0,0,0
7,88.7516,1,9,1,1,0,0,0,0
8,61.4361,1,10,1,0,0,0,0,1
9,63.8114,1,11,1,0,0,0,0,1


# **OLS**

In [124]:
#Modelo OLS

basic  = @formula(outcome ~ (treatment + school + female + mother_secondary + father_secondary + failed_one_year + cash_transfer + computer))
basic_results  = lm(basic, data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

outcome ~ 1 + treatment + school + female + mother_secondary + father_secondary + failed_one_year + cash_transfer + computer

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)       58.5095      0.368371     158.83    <1e-99  57.7875      59.2316
treatment          4.51228     0.230809      19.55    <1e-83   4.05987      4.96469
school            -0.00414664  0.000500685   -8.28    <1e-15  -0.00512805  -0.00316524
female             3.02781     0.234285      12.92    <1e-37   2.56859      3.48704
mother_secondary   1.01695     0.268085       3.79    0.0001   0.491468     1.5424

# **Double Lasso**

In [96]:
# Seat values for Lasso

lasso_model = fit(LassoModel, term(:outcome) ~  sum(term.(names(data[!, Not(["outcome","treatment"])]))), data; α = 0.8)
r_Y = residuals(lasso_model)
r_Y = DataFrame([r_Y], [:r_Y])

# Part. out d

lasso_model = fit(LassoModel, term(:treatment) ~  sum(term.(names(data[!, Not(["outcome", "treatment"])]))), data;  α = 0.8)
r_D = residuals(lasso_model)
r_D = DataFrame([r_D], [:r_D])

# ols 
data_aux = [r_Y r_D]
fm_1 = @formula(r_Y ~ r_D)
partial_lasso_fit = lm(fm_1, data_aux)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

r_Y ~ 1 + r_D

Coefficients:
───────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  9.67487e-16    0.115231   0.00    1.0000  -0.225868   0.225868
r_D          4.51234        0.230753  19.55    <1e-83   4.06003    4.96464
───────────────────────────────────────────────────────────────────────────

In [104]:
# output: estimated regression coefficient corresponding to the target regressor
est_lasso = coef(partial_lasso_fit)[2]
# Regress residuales
partial_lasso_fit = lm(fm_1, data_aux)
partial_lasso_est = coef(partial_lasso_fit)[2]

println("Coefficient for D via partialling-out using lasso is: ", partial_lasso_est )

Coefficient for D via partialling-out using lasso is: 4.512336363218052


# Double Lasson using HDMJL

In [166]:
outcome = data[:, "outcome"] |> collect
treatment = data[:, "treatment"] |> collect
school = data[:, "school"] |> collect
female = data[:, "female"] |> collect
mother_secondary = data[:, "mother_secondary"] |> collect
father_secondary = data[:, "father_secondary"] |> collect
failed_one_year = data[:, "failed_one_year"] |> collect
cash_transfer = data[:, "cash_transfer"] |> collect
computer = data[:, "computer"] |> collect
matrix_to_use = Matrix(select(data,Not(["outcome"])))
println("")

In [171]:
res_Y_0 = rlasso(matrix_to_use, outcome, post=false )

Dict{String, Any} with 15 entries:
  "tss"          => 3.21075e6
  "dev"          => [-2.24373, -29.836, -9.35185, -15.816, 12.4142, 21.148, 29.…
  "model"        => [1.0 3.0 … 0.0 0.0; 0.0 4.0 … 0.0 0.0; … ; 0.0 459.0 … 0.0 …
  "loadings"     => [7.03095, 3258.38, 7.03407, 7.02577, 6.97035, 6.15023, 6.57…
  "sigma"        => 14.0511
  "lambda0"      => 860.507
  "lambda"       => [6050.19, 2.80386e6, 6052.87, 6045.73, 5998.04, 5292.32, 56…
  "intercept"    => 58.4551
  "iter"         => 4
  "residuals"    => [-8.68649, -31.5496, -14.7452, -16.3857, 10.1363, 18.4307, …
  "rss"          => 2.92102e6
  "index"        => Bool[1, 1, 1, 1, 1, 1, 1, 1]
  "beta"         => [3.71294, -0.00248818, 2.18388, 0.602433, 0.411276, -6.0607…
  "options"      => Dict{String, Any}("intercept"=>true, "post"=>false, "meanx"…
  "coefficients" => [58.4551, 3.71294, -0.00248818, 2.18388, 0.602433, 0.411276…

# Double Lasson using HDMJL with partialling out

In [170]:
matrix_to_use_2 = Matrix(select(data,Not(["outcome","treatment"])))
res_Y_1 = rlassoEffect(matrix_to_use_2, outcome, treatment, method = "partialling out" )

Dict{String, Any} with 9 entries:
  "alpha"            => 4.50286
  "t"                => 19.5335
  "se"               => 0.230519
  "coefficients_reg" => [60.7908, -0.0043728, 3.15416, 1.02396, 0.856549, -6.59…
  "sample_size"      => 14796
  "coefficient"      => 4.50286
  "selection_index"  => Any[true, true, true, true, true, true, true]
  "residuals"        => Dict("v"=>[0.488984, -0.511016, 0.488984, -0.511016, -0…
  "coefficients"     => 4.50286

# Plotting

In [173]:
#1 

x = ["OLS","Double Lasso","HDM Lasso","HDM partialling out"]
y = [coef(basic_results)[2],partial_lasso_est,res_Y_0["coefficients"][2],res_Y_0["coefficients"][1]]

4-element Vector{Float64}:
  4.512280849760991
  4.512336363218052
  3.71293983038266
 58.45508671330132